In [1]:
import fio.database
import mip

db = fio.database.load_db("data")

In [2]:
import functools

def is_fluid(name):
    return db.fluid.query(f"""map(select(.name == "{name}"))|any""")

def is_item(name):
    return db.item.query(f"""map(select(.name == "{name}"))|any""")

assert is_item("iron-ore")
assert not is_fluid("iron-ore")

assert not is_item("water")
assert is_fluid("water")

@functools.lru_cache(16)
def get_assembly_machines(category, craftable_only=True):
  ms = []

  for m in db.assembling_machine.find(f""".[] | select(.crafting_categories."{category}")"""):
    if craftable_only and not db.recipe.query(f"""map( select(.main_product.name == "{m['name']}" )) | any"""):
      continue
    ms.append(m)

  for m in db.furnace.find(f""".[] | select(.crafting_categories."{category}") """):
    if craftable_only and not db.recipe.query(f"""map( select(.main_product.name == "{m['name']}" )) | any"""):
      continue
    ms.append(m)

  return ms

# assert get_assembly_machines("nuclear-fusion") == ['kr-fusion-reactor']
# assert set(get_assembly_machines("basic-crafting")) == {'assembling-machine-1', 'assembling-machine-2', 'assembling-machine-3', 'kr-advanced-assembling-machine'}

assert len(db.recipe.find(".[] | select(.hidden) | .name")) == 750
assert len(db.assembling_machine.find(""".[] | select(.crafting_categories."basic-crafting") | .name""")) == 6
assert db.recipe.query("""map( select(.main_product.name == "laser-turret" ))|any""") == True
assert db.recipe.query("""map( select(.main_product.name == "rien" ))|any""") == False

In [3]:
recipes = fio.database.HyperDiGraph()

ignore_machines = [
    "kr-advanced-assembling-machine",
    "kr-advanced-chemical-plant",
    "kr-advanced-furnace",
    "electric-furnace",
    "assembling-machine-3",
    "kr-matter-assembler",
    "kr-matter-plant",
    "kr-quantum-computer"
]

for r in db.recipe.find(".[]"):
    for m in get_assembly_machines(r['category']):

        if m["name"] in ignore_machines:
            continue

        A = {i["name"] for i in r["ingredients"]}
        B = {p["name"] for p in r["products"]}
        recipes.add_edge(A, B, {"recipe": r, "machine": m, "name": f"{r['name']}#{m['name']}"})


In [4]:
import fio.flow


source = {
    "iron-ore": float("+inf"),
    "crude-oil": float("+inf"),
    "copper-ore": float("+inf"),
    "stone": float("+inf"),
    "coal": float("+inf"),
    "water": float("+inf"),
    "wood": float("+inf"),
    "biomass": float("+inf"),
    "uranium-238": float("+inf"),
    "imersite-powder": float("+inf"),
    "mineral-water": float("+inf"),
    "raw-rare-metals": float("+inf"),
}

target = {
    k: 1 for k in {
        # 'basic-tech-card',
        # 'advanced-tech-card',
        'automation-science-pack',
        # 'biters-research-data',
        'chemical-science-pack',
        'logistic-science-pack',
        'production-science-pack',
        'utility-science-pack',
        # 'matter-research-data',
        # 'matter-tech-card',
        # 'military-science-pack',
        # 'singularity-tech-card',
    }
}

model = fio.flow.optimize_model(recipes, dict(**source, **{k: -v for k, v in target.items()}))

print(model["status"])
# model = build_detailed_graph(model)

OptimizationStatus.OPTIMAL


In [5]:

from collections import defaultdict, namedtuple

inputs = defaultdict(float)
outputs = defaultdict(float)
c = defaultdict(int)


flows = []
i = 0


from uuid import uuid4

Cell = namedtuple("Cell", ["id", "recipe", "machine", "intergral", "fractional"])

def unique_id(i):
  r = unique_id.c[i]
  unique_id.c[i] += 1
  return f"id_{i}_{r}"
unique_id.c = defaultdict(int)

import networkx as nx
graph = nx.DiGraph()

per_rows = 4
for r, m, k, f in [tuple(k.split("#") + list(v)) for k, v in model["flow"].items()]:
    c[m] += k

    r = db.recipe.query(f"""map( select(.name == "{r}"))""")[0]

    m1 = db.assembling_machine.query(f"""map( select(.name == "{m}")) """)
    m2 = db.furnace.query(f"""map( select(.name == "{m}")) """)
    m = (m1 + m2)[0]

    for _ in range(k-1):
      i = unique_id(r["name"])
      flows.append(Cell(i, r, m, 1, 1))
      graph.add_node(i, recipe=r, machine=m, k=1, f=1)

    assert f-k+1 > 0
    flows.append(Cell(unique_id(r["name"]), r, m, 1, f-k+1))
    graph.add_node(i, recipe=r, machine=m, k=1, f=f-k+1)

c

defaultdict(int,
            {'assembling-machine-2': 151,
             'steel-furnace': 117,
             'kr-atmospheric-condenser': 6,
             'chemical-plant': 8,
             'oil-refinery': 13,
             'kr-crusher': 6,
             'kr-research-server': 8,
             'kr-filtration-plant': 5,
             'kr-fuel-refinery': 16})

# Second step.


We now need to know how many machines are needed for which exchange. To do so, we build an other linear program, with pack bins everywhere.




# Bin packing

Inputs:
* $v[j]$ output flow required
* $u[i]$ input flow required

Variables:
* $x[i, j]$ real between 0 and 1, indicate the flow from $i$ to j$
* $y[i, j]$ boolean indicate if the edge from $i$ to $j$

Minimize $\displaystyle \sum_{i, j} y[i, j]$
Subject:
* $\displaystyle \forall i j, x[i, j] \leq y[i, j]$
* $\displaystyle \forall i, \sum_j x[i, j] = u_i$
* $\displaystyle \forall j, \sum_i x[i, j] = v_j$




In [6]:
import fio.binpacking
from fractions import Fraction

items = set()

flow_in = defaultdict(list)
flow_out = defaultdict(list)
for f in flows:

  ing = {k: v*f.fractional for k, v in fio.flow.ingredients_coefs(f.recipe, f.machine).items()}
  pro = {k: v*f.fractional for k, v in fio.flow.products_coefs(f.recipe, f.machine).items()}

  for i, q in ing.items():
    q = Fraction(q).limit_denominator()
    assert q >= 0
    flow_in[i].append((f.id, q))
  for i, q in pro.items():
    q = Fraction(q).limit_denominator()
    assert q >= 0
    flow_out[i].append((f.id, q))


for i in set(flow_in.keys()) | set(flow_out.keys()):
  a = sum(q for _, q in flow_in[i])
  b = sum(q for _, q in flow_out[i])
  if a > b:
    flow_out[i].append(((unique_id("sink")), a - b))
  elif b > a:
    flow_in[i].append(((unique_id("source")), b - a))


assert set(flow_in.keys()) == set(flow_out.keys())

In [7]:
for k in flow_in.keys():
  in_ = flow_in[k]
  out = flow_out[k]

  idxA, A = zip(*out)
  idxB, B = zip(*in_)

  pairs = fio.binpacking.optimize([a / sum(A) for a in A], [b / sum(B) for b in B])

  for (i, j), f in pairs.items():
    graph.add_edge(idxA[i], idxB[j], w=f/A[i]*sum(A))

In [47]:
import random
from collections import deque

flatten = nx.Graph(graph)

nx.normalized_laplacian_spectrum(flatten)

<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: normalized_laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


array([-2.22044605e-15,  1.12388310e-01,  1.29014750e-01,  1.51210386e-01,
        1.80747373e-01,  2.07977352e-01,  2.23339780e-01,  2.32072462e-01,
        2.54730982e-01,  2.75486697e-01,  2.87889163e-01,  2.89576420e-01,
        2.94429217e-01,  3.06484189e-01,  3.10207605e-01,  3.17987463e-01,
        3.23273256e-01,  3.27262940e-01,  3.36684476e-01,  3.37407819e-01,
        3.43102061e-01,  3.48373421e-01,  3.55054748e-01,  3.59601421e-01,
        3.64026690e-01,  3.72513809e-01,  3.76894030e-01,  3.78039173e-01,
        3.78960159e-01,  3.83872542e-01,  3.99421049e-01,  4.01010433e-01,
        4.04326791e-01,  4.08933025e-01,  4.13388984e-01,  4.23671598e-01,
        4.29670465e-01,  4.33132450e-01,  4.36732187e-01,  4.42560846e-01,
        4.48466952e-01,  4.53533920e-01,  4.58740816e-01,  4.60423209e-01,
        4.63688796e-01,  4.71466557e-01,  4.75838206e-01,  4.81734523e-01,
        4.86378696e-01,  4.89666772e-01,  5.00304150e-01,  5.04808587e-01,
        5.13958118e-01,  

In [43]:


A = {n for n in flatten.nodes if random.random() > 0.5}
B = {n for n in flatten.nodes if n not in A}
h
aa = defaultdict(set)
alevel = dict()
bb = defaultdict(set)
blevel = dict()

for u in flatten.nodes:
  k = 0
  for v in flatten[u]:
    if (u in A and v in B) or (u in B and v in A):
      k += 1
    if (u in A and v in A) or (u in B and v in B):
      k -= 1
  if u in A:
    aa[k].add(u)
    alevel[u] = k
  elif u in B:
    bb[k].add(u)
    blevel[u] = k

best_cut = (2*sum(alevel[u] for u in flatten.nodes if u in alevel and alevel[u] > 0), A)
current_cut = best_cut[0]

finished = set()
while True:
  try:
    while True:
      k = max(aa.keys())
      if len(aa[k]) == 0:
        del aa[k]
      else:
        break
  except ValueError:
    break

  u = random.choice(list(aa[k]))
  assert u in A
  aa[k].discard(u)
  del alevel[u]
  current_cut -= k
  blevel[u] = - k
  A.discard(u)
  B.add(u)
  best_cut = min(best_cut, (current_cut, A))
  finished.add(u)
  for v in flatten[u]:
    if v in finished:
      continue
    if v in A:
      j = alevel[v]
      alevel[v] = j + 1
      aa[j].discard(v)
      aa[alevel[v]].add(v)
    if v in B:
      j = blevel[v]
      blevel[v] = j - 1
      bb[j].discard(v)
      bb[blevel[v]].add(v)

  try:
    while True:
      k = max(bb.keys())
      if len(bb[k]) == 0:
        del bb[k]
      else:
        break
  except ValueError:
    break

  u = random.choice(list(bb[k]))
  assert u in B
  bb[k].discard(u)
  del blevel[u]
  alevel[u] = - k
  current_cut -= k
  B.discard(u)
  A.add(u)
  best_cut = min(best_cut, (current_cut, A))
  finished.add(u)
  for v in flatten[u]:
    if v in finished:
      continue
    if v in A:
      j = alevel[v]
      alevel[v] = j - 1
      aa[j].discard(v)
      aa[alevel[v]].add(v)
    if v in B:
      j = blevel[v]
      blevel[v] = j + 1
      bb[j].discard(v)
      bb[blevel[v]].add(v)


In [46]:
size, A = best_cut
B = {n for n in flatten.nodes if n not in A}

print(size, len(A), len(B))

sum(((u in A and v in B) or (u in B and v in A)) for u, v in flatten.edges)


19 178 169


442

In [9]:
# nx.draw(graph, pos=nx.kamada_kawai_layout(graph), node_size=100, alpha=.5)